In [70]:
import json
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
import tweepy
from tweepy import OAuthHandler
import datetime
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os.path

now = datetime.datetime.now()

In [71]:
consumer_key = "33xt8At2xCpcmpeWiF3L19wBd"
consumer_secret = "oEU0f9BayuBoy486gLO1qHBbNj2WhuT3JzOzPv7vfg1BkImTcF"
access_token = "2573531050-0xyxOmpeAzvEldrjVr0fBE0D8r0zx6tAXqdowQX"
access_token_secret = "CWvNa2iK7u5SFpyiCqIyIiBTuDOgD8uCPv2M1gOFYsvoA"

In [72]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [73]:
def get_tweets(user_id, n_tweets = 40):
    
    '''Gets specific number of tweets from a given user'''
    
    tweets = api.user_timeline(screen_name=user_id, 
                               # 200 is the maximum allowed count
                               count=n_tweets,
                               include_rts = False,
                               # Necessary to keep full_text 
                               # otherwise only the first 140 words are extracted
                               tweet_mode = 'extended'
                               )
    return tweets

In [77]:
def extract_tweet_info(tweets):
    
    '''Takes a tweepy results set and extracts information from it; returns a df'''
    
    urls = []
    htags = []
    mentions = []
    encode_text = []
    creation_date = []
    favorites = []
    rts = []
    
    # Loop through tweets to extract important info
    for tweet in tweets:
        urls.append(re.findall('(?P<url>https?://[^\s]+)', tweet.full_text))
        htags.append(re.findall(r"#(\w+)", tweet.full_text))
        mentions.append(re.findall(r"@(\w+)", tweet.full_text))
        encode_text.append(tweet.full_text.encode("utf-8").decode("utf-8").replace(f'\n',' '))
        favorites.append(tweet.favorite_count)
        rts.append(tweet.retweet_count)
        creation_date.append(tweet.created_at)
    
    # Separate urls into resource or tweet
    tweet_urls = []
    resource_urls = []

    for url in urls:
        if len(url) == 1:
            tweet_urls.append(url[0])
            resource_urls.append('')
        else:
            tweet_urls.append(url[1])
            resource_urls.append(url[0])
            
    # Search for specific keywords of interest
    evacs = [bool(re.search(r"evac(\w+)", a))*1 for a in encode_text]
    red_flags = [bool(re.search(r"red flag(\w+)", a))*1 for a in encode_text]
    new_incidents = [bool(re.search(r"new incident(\w+)", a))*1 for a in encode_text]
    
    # Move to df and rename cols
    tweets_df = pd.DataFrame([creation_date, htags, mentions, favorites, rts, encode_text, resource_urls, tweet_urls,
                              evacs, red_flags, new_incidents]).T
    tweets_df.columns = ['time_tweeted','hashtags','mentions','favorites','retweets','tweet_text','resource_url',
                         'tweet_url','is_evac','is_red_flag','is_new_incident']
    
    return tweets_df

In [75]:
def send_email(email_recipient,
               email_subject,
               email_message,
               attachment_location = ''):

    email_sender = 'tficarrr@gmail.com'

    msg = MIMEMultipart()
    msg['From'] = email_sender
    msg['To'] = email_recipient
    msg['Subject'] = email_subject

    msg.attach(MIMEText(email_message, 'html'))

    if attachment_location != '':
        filename = os.path.basename(attachment_location)
        attachment = open(attachment_location, "rb")
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',
                        "attachment; filename= %s" % filename)
        msg.attach(part)

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.login('tficarrr@gmail.com', 't2tdog1313?')
    text = msg.as_string()
    server.sendmail(email_sender, email_recipient, text)
    print('email sent')
    server.quit()
    return

In [78]:
if __name__ == '__main__':
    
    user_id = 'cal_fire'
    tweets = get_tweets(user_id, 30)
    tweets_df = extract_tweet_info(tweets)
    email_body = """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(tweets_df.to_html())
    send_email('tficar@yahoo.com','Fire Tweets',email_body)

email sent
